# Neptune acceptance notebook

This notebook runs high-level acceptance tests and demonstrations for the Neptune Testbed systems. Each test measures various photon rates and fidelities in the system, compares these to the acceptance criteria, and returns PASS or FAIL for each test. The Neptune System has three different types of quantum light source available to the user. This notebook tests all three. The other acceptance criteria such as g2 etc. are presented in accompanying acceptance appendices. 

This notebook requires the ORCA SDK to be installed on the computer running this notebook.

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
import sys


# from ptseries.tbi import create_tbi

### Acceptance criteria

The following cells define the acceptance critera for each quantum light source, as laid out in the sytem test and validation document.

In [ ]:
qd_acceptance_targets = {"ACCEPTANCE_RATES_1_PHOTON": 300, # Min rate at which we measure 1-photon events
                        "ACCEPTANCE_RATES_2_PHOTON": 30, # Min rate at which we measure 2-photon events
                        "ACCEPTANCE_RATES_3_PHOTON": 3, # Min rate at which we measure 3-photon events
                        "ACCEPTANCE_RATES_4_PHOTON": 1,  # Min rate at which we measure 4-photon events
                        "ACCEPTANCE_RATES_5_PHOTON" : 1, # Min rate at which we measure 5-photon events
                        "ACCEPTANCE_ACCURACY_1_PHOTON": 90, #Percentage of 1000 states measured when input state is 1000 and identity is applied
                        "ACCEPTANCE_ACCURACY_1_PHOTON_ROUTING:": 60, # Percentage of 0001 states measured when input state is 1000 and swaps are applied
                        "ACCEPTANCE_ACCURACY_2_PHOTON": 40, # Percentage of 1100 states measured when input state is 1100 and identity is applied
                        }

hsps_acceptance_targets = {"ACCEPTANCE_RATES_1_PHOTON": 300,
                        "ACCEPTANCE_RATES_2_PHOTON": 30,
                        "ACCEPTANCE_RATES_3_PHOTON": 3,
                        "ACCEPTANCE_RATES_4_PHOTON": 0.1,
                        "ACCEPTANCE_ACCURACY_1_PHOTON": 90,
                        "ACCEPTANCE_ACCURACY_1_PHOTON_ROUTING:": 60,
                        "ACCEPTANCE_ACCURACY_2_PHOTON": 40,
                        }

smsv_acceptance_targets = {"ACCEPTANCE_RATES_1_PHOTON": 300,
                        "ACCEPTANCE_RATES_2_PHOTON": 30,
                        "ACCEPTANCE_RATES_3_PHOTON": 3,
                        "ACCEPTANCE_RATES_4_PHOTON": 1,
                        "ACCEPTANCE_RATES_5_PHOTON": 1,
                        }


# Notebook Set-Up

In the following cells we call-out some settings required for further on in the notebook and instantiate two instances of the PT System, one for the HSPS and SMSV and a second for the QD lgiht source. We also define some helper functions here for some simple plots of the data returned from the machine.

Here, we set how many samples are collected for the different number of photons. As the rates decrease with higher photon numbers, we collect fewer samples for higher photon numbers. Due to sampling overheads we need to ensure we ask for a significantly large number of samples for jobs that return quickly to avoid the results being skewed. 

In [ ]:
N_SAMPLES = {
    1: 10000,  # Number of samples to collect for 1 photon
    2: 2000,  # Number of samples to collect for 2 photons
    3: 200,  # Number of samples to collect for 3 photons
    4: 50,  # Number of samples to collect for 4 photons
    5: 10, # Number of samples to collect for 5 photons
}

In the following cell we instantiate two instances of the PT System hardware, please change the url to match the IP addresses of the target machines. In the Neptune system, the smsv source and hsps source are accessed at the same IP address and use the same underlying hardware. The quantum dot source is accessed at a separate IP address. Note that you may need to append the port number ":8080"

In [ ]:
qd_tbi_params = {
    "tbi_type": "PT",
    "url": "http://192.168.22.197:8080",
    "gbs": False,
    "postselection": True,
    "postselection_threshold": None # None == strong postselection
}

gbs_tbi_params = {
    "tbi_type": "PT",
    "url": "http://192.168.22.198:8080",
    "gbs": True,
    "postselection": True,
    "postselection_threshold": None # None == strong postselection
}
hsps_tbi_params = {
    "tbi_type": "PT",
    "url": "http://192.168.22.198:8080",
    "gbs": False,
    "postselection": True,
    "postselection_threshold": None # None == strong postselection
}

In the next cell we create two helper functions, one to estimate the photon rates returned by our sampling jobs and a second to plot the sample distributions returned.

In [ ]:
def estimate_statistics(tbi, input_state, theta_list, n_samples, overhead_time_with_margin):
    """This function runs an experiment on a particular tbi and
    returns the number of samples effectively collected, the rates, the accuracy,
    the time taken to collect the samples, and the samples themselves"""

    start_time = time.time()
    samples = tbi.sample(
        input_state=input_state, theta_list=theta_list, n_samples=n_samples, 
    )
    request_time = max([time.time() - start_time - overhead_time_with_margin, 0.1])

    rates = n_samples / request_time
    target_state = samples.get(input_state, 0)
    accuracy = 100 - 100 * (n_samples - target_state) / n_samples

    return n_samples, rates, accuracy, request_time, samples


def plot(
    samples,
    show_plot=False,
    title="Counts Data",
):
    ## Create a bar plot
    keys = ["".join(map(str, key)) for key in samples.keys()]
    values = list(samples.values())
    fig = plt.figure(figsize=(6, 4), dpi=200)
    plt.bar(keys, values)
    plt.xticks(rotation=90, fontsize=4)
    # Add labels and title
    plt.xlabel("Output state")
    plt.ylabel("Occurences")
    plt.title(f"{title}")
    ## Show the plot
    if show_plot:
        plt.show()

# A1: Quantum Dot Acceptance Test

The following cells will test the performance of the Quantum Dot light source Boson Sampling System. 

In [ ]:
tbi = create_tbi(tbi_params = qd_tbi_params)  # SDK version 2.5.6 or greater
acceptance_targets = qd_acceptance_targets

This cell runs a single sample request for 1 photon. This eliminates any overheads due to re-calibration of the quantum processors or just in time compilation.

In [ ]:
tbi.sample(input_state=(1,0), theta_list=[0])

The following cell estimates any additional overheads for sampling jobs, which we account for in our rates calculations

In [ ]:
start_time = time.time()
tbi.sample(input_state=(1, 0), theta_list=[0])
overhead_time = time.time() - start_time
overhead_time_with_margin = max([overhead_time - 1, 0])

print(f"Overhead time for QD sampling is {overhead_time:.2f}s")
print(f"Reducing to {overhead_time_with_margin:.2f}s to leave a margin of error")

## A1.1 Photon rate and accuracy acceptance tests

All the acceptance tests run using the circuit below, shown here with a 1000 input state.

In [ ]:
tbi.draw(input_state=(1,0,0,0))

The following cell iterates over the [1,0,0,0] to [1,1,1,1] states and looks at the measured detection rates for each case. Then it compares these with the acceptance thresholds provided above.

In [ ]:
N_MIN_PHOTON = 1
N_MAX_PHOTON = 5

for n_photons in np.arange(N_MIN_PHOTON,N_MAX_PHOTON+1):

    # create tbi to apply postselecttion appropriately

    input_state = tuple([1]*n_photons)
    theta_list = [0]*(n_photons-1)
    output = estimate_statistics(
        tbi = tbi,
        input_state = input_state,
        theta_list = theta_list,
        n_samples = N_SAMPLES[n_photons],
        overhead_time = overhead_time_with_margin
    )

    n_samples_collected = output[0]
    rates = output[1]
    accuracy = output[2]
    request_time = output[3]
    samples = output[4]

    if n_photons == 1:
        samples_1_photons = samples

    print("============= {n_photons} Photon Data ======================")
    print(f"{n_samples_collected} samples collected in {request_time:.2f} seconds")
    print(f"{n_photons} photon rate is {rates:.3f} Hz")

    if rates > acceptance_targets[f"ACCEPTANCE_RATES_{n_photons}_PHOTON"]:
        print(f"Acceptance test passed for {n_photons} photon rate: PASS")
    else:
        print(f"Acceptance test passed for {n_photons} photon rate: FAIL")

    if "ACCEPTANCE_ACCURACY_{n_photons}_PHOTON" in acceptance_targets.keys():
        if accuracy > acceptance_targets["ACCEPTANCE_ACCURACY_{n_photons}_PHOTON"]:
            print("Acceptance test passed for 2 photon accuracy: PASS")
        else:
            print("Acceptance test passed for 2 photon accuracy: FAIL")

In [ ]:
plot(samples_1_photons)

## A1.2 One photon routing acceptance test
Sending input state [1,0,0,0] and all beam splitters to pi/2, this will route the photon from the first qumode to the last qumode in the sample. We measure how often we get target state [0,0,0,1]

In [ ]:
samples_routing = tbi.sample(
    input_state=(1, 0, 0, 0), theta_list=3 * [np.pi / 2], n_samples=N_SAMPLES[1]  # Sets all circuit elements to swap
)

n_samples_routing = sum(samples_routing.values())
n_target_samples = samples_routing.get((0,0,0,1), 0)
routing_accuracy = 100*n_target_samples/n_samples_routing

print("============= Photon Routing Data ======================")
print(f"Photon routing accuracy is {routing_accuracy:.0f} %")
plot(samples_routing)

In [ ]:
if routing_accuracy > acceptance_targets["ACCEPTANCE_ACCURACY_1_PHOTON_ROUTING"]:
    print("Acceptance test passed for 1 photon routing accuracy: PASS")
else:
    print("Acceptance test passed for 1 photon routing accuracy: FAIL")

# A2 Heralded Single Photon Source Acceptance Tests
The following cells will test the performance of the Heralded single-photon source light source Boson Sampling System.

In [ ]:
tbi = create_tbi(tbi_params = hsps_tbi_params)  # SDK version 2.5.6 or greater
acceptance_targets = hsps_acceptance_targets

This cell runs a single sample request for 1 photon. This eliminates any overheads due to re-calibration of the quantum processors or just in time compilation.

In [ ]:
tbi.sample(input_state=(1,0), theta_list=[0])

The following cell estimates any additional overheads for sampling jobs, which we account for in our rates calculations

In [ ]:
start_time = time.time()
tbi.sample(input_state=(1, 0), theta_list=[0])
overhead_time = time.time() - start_time
overhead_time_with_margin = max([overhead_time - 1, 0])

print(f"Overhead time for QD sampling is {overhead_time:.2f}s")
print(f"Reducing to {overhead_time_with_margin:.2f}s to leave a margin of error")

## A2.1 One photon rate and accuracy acceptance tests

All the acceptance tests run using the circuit below, shown here with a 1000 input state.

In [ ]:
tbi.draw(input_state=(1,0,0,0))

In [ ]:
input_state_1_photons = (1,0,0,0)
theta_list_1_photons = [0,0,0]  # Sets the circuit to identity
n_samples_1_photons = N_SAMPLES[1]

outputs_1_photons = estimate_statistics(
    tbi = tbi,
    input_state = input_state_1_photons,
    theta_list = theta_list_1_photons,
    n_samples = n_samples_1_photons,
    overhead_time = overhead_time_with_margin
)

n_samples_collected_1_photons = outputs_1_photons[0]
rates_1_photons = outputs_1_photons[1]
accuracy_1_photons = outputs_1_photons[2]
request_time_1_photons = outputs_1_photons[3]
samples_1_photons = outputs_1_photons[4]

print(f"{n_samples_1_photons} samples collected!")

In [ ]:
print("============= 1 Photon Data ======================")
print(f"{n_samples_collected_1_photons} samples collected in {request_time_1_photons:.2f} seconds")
print(f"1 photon rate is {rates_1_photons:.0f} Hz")
print(f"1 photon accuracy is {accuracy_1_photons:.0f} %")
plot(samples_1_photons)

In [ ]:
if rates_1_photons > hsps_acceptance_targets["ACCEPTANCE_RATES_1_PHOTON"]:
    print("Acceptance test passed for 1 photon rate: PASS")
else:
    print("Acceptance test passed for 1 photon rate: FAIL")

if accuracy_1_photons > hsps_acceptance_targets["ACCEPTANCE_ACCURACY_1_PHOTON"]:
    print("Acceptance test passed for 1 photon accuracy: PASS")
else:
    print("Acceptance test passed for 1 photon accuracy: FAIL")

## A2.2 One photon routing acceptance test
Sending input state [1,0,0,0] and all beam splitters to pi/2, this will route the photon from the first qumode to the last qumode in the sample. We measure how often we get target state [0,0,0,1]

In [ ]:
samples_routing = tbi.sample(
    input_state=(1, 0, 0, 0), theta_list=3 * [np.pi / 2], n_samples=N_SAMPLES[1]  # Sets all circuit elements to swap
)

n_samples_routing = sum(samples_routing.values())
n_target_samples = samples_routing.get((0,0,0,1), 0)
routing_accuracy = 100*n_target_samples/n_samples_routing

print("============= Photon Routing Data ======================")
print(f"Photon routing accuracy is {routing_accuracy:.0f} %")
plot(samples_routing)

In [ ]:
if routing_accuracy > hsps_acceptance_targets["ACCEPTANCE_ACCURACY_1_PHOTON_ROUTING"]:
    print("Acceptance test passed for 1 photon routing accuracy: PASS")
else:
    print("Acceptance test passed for 1 photon routing accuracy: FAIL")

## A2.3 Two, Three and Four Photon Rate Acceptance Test
The following cells evaluate the rate of two-photon samples returned by the machine.

In [ ]:
N_MIN_PHOTON = 2
N_MAX_PHOTON = 4

for n_photons in np.arange(N_MIN_PHOTON,N_MAX_PHOTON+1):

    input_state = tuple([1]*n_photons)
    theta_list = [0]*(n_photons-1)
    output = estimate_statistics(
        tbi = tbi,
        input_state = input_state,
        theta_list = theta_list,
        n_samples = N_SAMPLES[n_photons],
        overhead_time = overhead_time_with_margin
    )

    n_samples_collected = output[0]
    rates = output[1]
    accuracy = output[2]
    request_time = output[3]
    samples = output[4]

    print("============= {n_photons} Photon Data ======================")
    print(f"{n_samples_collected} samples collected in {request_time:.2f} seconds")
    print(f"{n_photons} photon rate is {rates:.3f} Hz")

    if rates > acceptance_targets[f"ACCEPTANCE_RATES_{n_photons}_PHOTON"]:
        print(f"Acceptance test passed for {n_photons} photon rate: PASS")
    else:
        print(f"Acceptance test passed for {n_photons} photon rate: FAIL")

    if "ACCEPTANCE_ACCURACY_{n_photons}_PHOTON" in acceptance_targets.keys():
        if accuracy > acceptance_targets["ACCEPTANCE_ACCURACY_{n_photons}_PHOTON"]:
            print("Acceptance test passed for 2 photon accuracy: PASS")
        else:
            print("Acceptance test passed for 2 photon accuracy: FAIL")